Generate a Gaussian random matrix and check the semi-circle law

In [3]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import matplotlib.pylab as plt
import matplotlib
%matplotlib qt
from numpy import linalg as la
from scipy.optimize import fsolve
from scipy import linalg as scpla
import seaborn as sb
from mpl_toolkits.axes_grid1 import make_axes_locatable
from cmath import *
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import fsolve,leastsq 
from math import tanh,cosh
from sympy import *
extras_require = {'PLOT':['matplotlib>=1.1.1,<3.0']},


In [4]:
# generate Gaussian random matrix
nn = 1000 
x = np.random.randn(nn,nn)/np.sqrt(nn)
lvec,sv,rvech=la.svd(x)
n, bins, patches = plt.hist(sv, 50, density=True, facecolor='g', alpha=0.75)

In [5]:
# largest singular values and singular vectors phase transition
# define theta_, the threshold of the phase transition
def Dtransform_mux(z,c,xrange,mux='semicircle'):
    if mux=='semicircle':
        tt=xrange
        dtt=xrange[1]-xrange[0]
        muxpdf = 2*np.sqrt(4-tt**2)
        muxpdf/=np.sum(muxpdf*dtt)
        plt.plot(tt,muxpdf)
        deltamux=np.zeros_like(muxpdf)
        deltamux[1:]= np.diff(muxpdf)

        coeff=z/(z**2-tt**2)
        integralh=np.sum(coeff*deltamux)
        Dtrans=integralh*(c*integralh+(1-c)/z)

        return Dtrans
def Gtransform_mux(z,c,xrange,mux='semicircle'):
    if mux=='semicircle':
        tt=xrange
        dtt=xrange[1]-xrange[0]
        idxzeros=np.where(tt<=-2)
        idxzerosr=np.where(tt>=2)
        muxpdf = np.sqrt(4-tt**2)
        muxpdf[idxzeros]=0
        muxpdf[idxzerosr]=0
        muxpdf/=np.sum(muxpdf*dtt)
        # plt.plot(tt,muxpdf)
        deltamux=np.zeros_like(muxpdf)
        deltamux= np.sqrt(4-tt**2)/(2*np.pi)
        deltamux[idxzeros]=0
        deltamux[idxzerosr]=0
        deltamux/=np.sum(deltamux*dtt)
        deltamux[1:]= np.diff(muxpdf)
        deltamux[-1]=0
        plt.plot(tt,deltamux)
        coeff=1/(z-tt)
        Gtrans=np.sum(coeff*deltamux)
        

        return Gtrans
# here the supremum
eps = 1e-5
bsupremum=2
bsupremump=bsupremum+eps
eigvrange=np.linspace(-3.0,3.0,6000)

Gtran_supreplus=Gtransform_mux(bsupremump,1,eigvrange,mux='semicircle')
theta_=1/Gtran_supreplus
print('critical point:',theta_)

critical point: -0.02298084063167618


In [4]:
#
def rootz(z):
    solve1=((z**2-2)+np.sqrt((z**2-2)**2-4))/2.0
    solve2=((z**2-2)-np.sqrt((z**2-2)**2-4))/2.0
    return (solve1,solve2)
solve1,solve2=rootz(2.0001)
print('solve1,2:',solve1,solve2)

solve1,2: 1.0202012550109552 0.9801987549890456


In [18]:
# define low-rank structure
nn=600
JE,JI=1.6,1.2
J = np.zeros((nn*2,nn*2))
J[:,:nn],J[:,nn:]=JE/(nn),-JI/(nn)
lvec,sv,rvech=la.svd(J)
print('singular value:',sv[0])
theo_sv=np.sqrt(2*(JE**2+JI**2))
print('singular values theo:',theo_sv)


singular value: 2.828427124746185
singular values theo: 2.8284271247461903


In [19]:
# numerical singular values 
x = np.random.randn(2*nn,2*nn)/np.sqrt(2*nn)
Rlvec,Rsv,Rrvech=la.svd(x)
n, bins, patches = plt.hist(Rsv, 50, density=True, facecolor='g', alpha=0.75)

In [20]:
x_=x+J
lvec_,sv_,rvech_=la.svd(x_)
n_, bins, patches = plt.hist(sv_, 50, density=True, facecolor='g', alpha=0.75)
outlier=sv_[np.where(sv_>2.0)]
print('outlier:',sv_[np.where(sv_>2.0)])
print(np.max(n_))

outlier: [3.17557485]
0.6693186669081389


In [14]:
# a1 and a2
z=outlier
a1,a2=((z**2-2)+np.sqrt((z**2-2)**2-4))/2.0,((z**2-2)-np.sqrt((z**2-2)**2-4))/2.0
GCauchy=-(-z+z*(a1-1)**2/((a1-a2)*a1))/2.0
print('GCauchy:',1.0/GCauchy)

GCauchy: [2.79411848]


In [21]:
# varphi
varphi=z*a2/(a2+1)
overlap=-(1+1/a2)/sv[0]**2
overnum=np.sum(lvec[:,0]*lvec_[:,0])
print('overlap:',overlap,overnum)

overlap: [-1.10088726] 0.9372429689742006


In [ ]:
((z^2*9-(z^2+1)^2)*z)/(z+0.381966) at z=-0.381966